Code from A3 problem1-prompt.ipynb and https://github.com/SewoongLee/reproduce-llama3-arithmetic/blob/main/llama3-tutorial-gsm8k.ipynb

You need to set your OPENAI_API_KEY environment variable in .env

In [1]:
from openai import OpenAI
from tqdm import tqdm
import textwrap
import dotenv
import os
%load_ext dotenv
%dotenv

client=OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# we use gpt-4o-mini as default: current pricing is 
# $0.150 / 1M input tokens, $0.600 / 1M output tokens
def generate_gpt_response(prompt,messages=[], model="gpt-3.5-turbo", temperature=0.7):
    messages.append({"role": "user", "content": prompt})
            
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature = temperature
    )
    content=response.choices[0].message.content
    messages.append({'role': 'assistant', 'content': content})
    return messages


def print_message(message, width=80):
    text=message["role"]+": " + message["content"]
    wrapped_text = textwrap.fill(text, width=width)
    
    for line in wrapped_text.split('\n'):
        print(f" {line.ljust(width)} ")        
    

cannot find .env file


load ablated gsm8k json files

In [ ]:
import os
import random
import json
import re

random.seed(56)

ablated_dataset = []
q_indices = []
folder_path = "gsm_template/output"

files = os.listdir(folder_path)
sorted_files = sorted(files, key=lambda x: int(x.split('--')[0][1:]))

print(sorted_files)
print()

repeat = 20

for file_name in sorted_files:
    f_path = os.path.join(folder_path, file_name)

    match = re.search(r'\d+', f_path)
    num = match.group(0)
    q_indices.append(int(num))

    with open(f_path, 'r') as f:
        lines = f.readlines()

    # random_lines = random.sample([json.loads(item.strip()) for item in lines], min(repeat, len(lines)))
    # print(random_lines)
    # print()

    # ablated_dataset.append(random_lines)
    ablated_dataset.append([json.loads(item.strip()) for item in lines])

print()
print(len(ablated_dataset))
print(len(ablated_dataset[0]))
print(q_indices)


['q0--NUM100.jsonl', 'q1--NUM100.jsonl', 'q2--NUM100.jsonl', 'q3--NUM100.jsonl', 'q4--NUM100.jsonl', 'q5--NUM100.jsonl', 'q6--NUM100.jsonl', 'q7--NUM100.jsonl', 'q8--NUM100.jsonl', 'q9--NUM100.jsonl', 'q11--NUM100.jsonl', 'q12--NUM100.jsonl', 'q13--NUM100.jsonl', 'q14--NUM100.jsonl', 'q15--NUM100.jsonl', 'q19--NUM100.jsonl', 'q20--NUM100.jsonl', 'q23--NUM100.jsonl', 'q24--NUM100.jsonl', 'q25--NUM100.jsonl']

[{'problem': 'Kc Otis sold clips to 48 of their friends in April, and then they sold 50% as many clips in May.How many clips did Kc Otis sell altogether in April and May?', 'result': '72', 'template_name': 0, 'idx': 71}, {'problem': 'Josh Joaquin sold clips to 48 of their friends in April, and then they sold 50% as many clips in May.How many clips did Josh Joaquin sell altogether in April and May?', 'result': '72', 'template_name': 0, 'idx': 1}, {'problem': 'Cristina Boogie sold clips to 48 of their friends in April, and then they sold 50% as many clips in May.How many clips did Cr

We will use the following system prompt

In [5]:
system_prompt= \
'''You are an expert math tutor specializing in grade-school-level problems like those in the GSM8K dataset. 
Solve each problem step by step, explaining the reasoning and calculations clearly and concisely to help the student understand the process. 
Please return your final answer after the delimiter \"####\" as a numeric number'''

We can answer from gpt api using fewshots in context learning and system prompt

In [6]:
import re

def extract_ans_from_response(answer: str, eos=None):
    if eos:
        answer = answer.split(eos)[0].strip()

    answer = answer.split('####')
    
    if len(answer) > 2:
        answer = answer[-2].strip()
    elif len(answer) == 2:
        answer = answer[-1].strip()
    else:
        answer = "NA"

    if isinstance(answer, str):
        for remove_char in [',', '$', '%', 'g']:
            answer = answer.replace(remove_char, '')

    try:
        return int(answer)
    except ValueError:
        return answer

# def extract_ans_from_response(answer: str, eos=None):
#     if eos:
#         answer = answer.split(eos)[0].strip()

#     answer = answer.split('####')[-1].strip()

#     for remove_char in [',', '$', '%', 'g']:
#         answer = answer.replace(remove_char, '')

#     try:
#         return int(answer)
#     except ValueError:
#         return answer

# def get_num_answer(text):
#     num_matches = re.findall(r'\d+\.?\d*', text)

#     if not num_matches:
#         return None
    
#     final_answer = num_matches[-1]

#     return final_answer

# Implement the function answer_from_gpt(question,temperature,num_shots), which should return the gpt answer as a string
# (see how it is called in the function evaluate_fewshots below)
# temperature is the temperature parameter passed to gpt api
# if num_shots>0, you should use the first num_shots examples in train_data for in-context learning
# Please use the system prompt provided above (see lecture10-gpt.ipynb for examples of system prompt usage)
#
def answer_from_gpt(question, temperature, num_shots):
    example = []

    # if num_shots > 0:
    #     for i in range(num_shots):
    #         curr_ex = train_data[i]
    #         example.append({"role": "user", "content": curr_ex['question']})
    #         example.append({"role": "assistant", "content": curr_ex['answer']})

    full_prompt = system_prompt + "\n" + question
    # print("Q: ", full_prompt)

    messages = generate_gpt_response(prompt=full_prompt, messages=example, temperature=temperature)
    answer = messages[-1]["content"]
    # print("A: ", answer)
    last_token = extract_ans_from_response(answer)
    return last_token

Evaluate on a small subset of gsm8k test data

In [ ]:
def evaluate_fewshots(data, temperature,num_shots):
    num_corrects=0
    for k in  tqdm(range(len(data))):
        gpt_answer=answer_from_gpt(data[k]['problem'],temperature=temperature,num_shots=num_shots)
        true_answer= data[k]['result']
        # print(f'k={k}: gpt_answer={gpt_answer}  true_answer={true_answer}')   # this is for debug only
        
        if gpt_answer != "NA":
            try:
                if float(gpt_answer) == float(true_answer):
                    # print("CORRECT")
                    num_corrects += 1
                else:
                    # print("INCORRECT")
                    num_corrects += 0
            except ValueError:
                # print("INCORRECT")
                num_corrects += 0
    
    accuracy = num_corrects/len(data)
    print(f'accuracy={accuracy:0.2f}')
    return accuracy

temperature = 0
num_shots = 0

total_inaccurate = 0

print(f'=== temperature={temperature} num_shots={num_shots} ===')
for i in range(len(ablated_dataset)):
    curr_ablation = ablated_dataset[i]

    print("Question ", q_indices[i])
    accuracy = evaluate_fewshots(curr_ablation, temperature, num_shots)
    if accuracy < 1:
        total_inaccurate += 1
    print()

print("Total Inaccurate: ", total_inaccurate)

=== temperature=0 num_shots=0 ===
Question  0


  5%|▌         | 1/20 [00:01<00:27,  1.44s/it]

k=0: gpt_answer=72  true_answer=72
CORRECT


 10%|█         | 2/20 [00:02<00:22,  1.26s/it]

k=1: gpt_answer=72  true_answer=72
CORRECT


...

k=17: gpt_answer=110  true_answer=110
CORRECT


 95%|█████████▌| 19/20 [00:24<00:01,  1.34s/it]

k=18: gpt_answer=110  true_answer=110
CORRECT


100%|██████████| 20/20 [00:26<00:00,  1.31s/it]

k=19: gpt_answer=110  true_answer=110
CORRECT
accuracy=1.00

Total Inaccurate:  8
